<a href="https://colab.research.google.com/github/SheidaTalei/FinalProject/blob/main/SheidaTalei_LSTM_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import regex
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import re
import string

In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import MaxPooling1D
from keras.layers import Conv1D


from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier



from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score



In [48]:
#------------------------------------------Empty Rows Removal----------------------------------------------------------------
# This function Will: 1- Remove all empty rows from csv file 2- Save data to the same csv
def removeEmptyRows(fileName):
    df = pd.read_csv(fileName, encoding='utf-8-sig')
    df = df.dropna(subset=['text'], how='all', axis=0) 
    df.to_csv(fileName, header=True, encoding='utf-8-sig',  index=False)
    
    return df

In [49]:
X_and_Y = removeEmptyRows ('/content/drive/MyDrive/Final/Prepared_train_Data.csv')

Y= X_and_Y.Label
X = X_and_Y.text

In [50]:
X_and_Y_test = removeEmptyRows('/content/drive/MyDrive/Final/Prepared_test_Data.csv')
Y_test= X_and_Y_test.Label
X_test = X_and_Y_test.text

In [51]:
#converting words to vectors
max_feature = 200
tokenizer = Tokenizer(num_words = max_feature, lower = True, split = ' ')
tokenizer.fit_on_texts(X )


X = tokenizer.texts_to_sequences(X)

#pad sequence is used to ensure that all sequences in a list have the same length.
X = pad_sequences(sequences = X, maxlen = max_feature, padding = 'pre')

In [52]:
def createLstmModel(units = 200, dropout = 0.5 ):
    
    
   #https://www.tensorflow.org/tutorials/text/text_classification_rnn
        
    #Adding Layers
    model = Sequential()
    model.add(Embedding(input_dim=300, output_dim=units))

    model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))  
   
    # model.add(LSTM(units = units , dropout = 0.2, return_sequences=True ))
    # model.add(Dense(units = units, activation="relu"))
    # model.add(Dropout(rate = dropout))


  
    model.add(LSTM(units = units , dropout = 0.2))
    model.add(Dense(units = units, activation="relu"))
    model.add(Dropout(rate = dropout))

    model.add(Dense(units = 1 ,activation="sigmoid"))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(), 'accuracy'])
#     model.fit(X, Y_train)
#     model.summary()
    return model


In [53]:
# Fit LSTM model
#source: https://apmonitor.com/do/index.php/Main/LSTMNetwork
# model = createLstmModel()
# history = model.fit(X, Y_train , epochs = 1)

# plt.figure()
# plt.ylabel('loss'); plt.xlabel('epoch')
# plt.semilogy(history.history['loss'])

In [54]:
model = KerasClassifier(build_fn=createLstmModel, epochs=1)


pipeline =  [
#         ('tokenizer', Tokenizer(num_words = max_feature, lower = True, split = ' ')),
#         ('pad', pad_sequences(sequences = X , maxlen = max_feature, padding = 'pre')),
        ('estimator', model)
    ]

parameters = {
#     'tokenizer__num_words': (100, 200, 300),
#     'pad__maxlen':( 200, 300, 400),
    
    'estimator__dropout': (0.5,0.1, 0.01),
    'estimator__units' :(100,  300, 700, 1000)
}

grid_search_tune = GridSearchCV(Pipeline(pipeline), parameters, cv=3, verbose=1)

LSTM = grid_search_tune.fit(X, Y)



Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


316/316 [==============================] - 6s 20ms/step - loss: 0.5498 - auc_109: 0.4966 - accuracy: 0.7664


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 14.1min finished


946/946 [==============================] - 10s 10ms/step - loss: 0.5439 - auc_110: 0.4919 - accuracy: 0.7726


In [55]:
print (grid_search_tune.best_estimator_.steps)

print ('Best Parameter for LSTM ',grid_search_tune.best_params_)
print('Best Score for LSTM: ',grid_search_tune.best_score_)

[('estimator', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7746a28ac8>)]
Best Parameter for LSTM  {'estimator__dropout': 0.5, 'estimator__units': 100}
Best Score for LSTM:  0.7725560069084167


In [56]:
max_feature = 200
tokenizer = Tokenizer(num_words = max_feature, lower = True, split = ' ')
tokenizer.fit_on_texts(X_test )


X = tokenizer.texts_to_sequences(X_test)

#pad sequence is used to ensure that all sequences in a list have the same length.
X = pad_sequences(sequences = X, maxlen = max_feature, padding = 'pre')

In [57]:
y_predict =grid_search_tune.predict(X)

In [58]:
print('Accuracy: ' , accuracy_score( Y_test,y_predict))
#Source: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
print('F1-score: ', f1_score(Y_test, y_predict))
#source: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
print('roc_auc_score: ', roc_auc_score(Y_test, y_predict))

Accuracy:  0.7743983073261042
F1-score:  0.0
roc_auc_score:  0.5
